In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from sklearn.datasets import make_classification
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from IPython import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from collections import defaultdict

import sklearn.datasets

torch.manual_seed(1)
np.random.seed(7)
sns.set(style="white", palette="muted", color_codes=True, context="talk")

%matplotlib inline
print(torch.__version__) 

1.6.0


In [114]:
import time

In [135]:
n_classes = 2

X, y = sklearn.datasets.make_classification(n_samples=1000,
                                            n_features=10,
                                            n_informative=5,
                                            n_redundant=2,
                                            n_repeated=0,
                                            class_sep=0.5,
                                            n_classes=n_classes,
                                            random_state = 4)

n_features = X.shape[1]

In [136]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=7)
print('len train:', len(X_train))
print('len test:', len(X_test))

len train: 800
len test: 200


In [137]:
BATCH_SIZE = 64

training_dataset = TensorDataset(torch.from_numpy(X_train).float(), 
                                 torch.from_numpy(y_train).long())
train_loader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

testing_dataset = TensorDataset(torch.from_numpy(X_test).float(), 
                                torch.from_numpy(y_test).long())
test_loader = DataLoader(testing_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [187]:
class Classifier(nn.Module):
    def __init__(self, n_features, n_hidden=256):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_classes),
            nn.LogSoftmax()
        )

    def forward(self, x):
        return self.network(x)

In [188]:
def grad_immediate_sensitivity(model, criterion, inputs, labels, epoch):
    inp = Variable(inputs, requires_grad=True)
    
    outputs = model.forward(inp)
    loss = criterion(torch.squeeze(outputs), labels)
    
    # (1) first-order gradient (wrt parameters)
    first_order_grads = torch.autograd.grad(loss, model.parameters(), retain_graph=True, create_graph=True)
    
    # (2) L2 norm of the gradient from (1)
    grad_l2_norm = torch.norm(torch.cat([x.view(-1) for x in first_order_grads]), p = 2) # CHANGE
    
    # (3) Gradient (wrt inputs) of the L2 norm of the gradient from (2)
    sensitivity_vec = torch.autograd.grad(grad_l2_norm, inp, retain_graph=True, create_graph=True)[0]
    
    # (4) L2 norm of (3) - "immediate sensitivity"
    s_norm = sensitivity_vec.norm(dim=1, p=2) # CHANGE

    immediate_sensitivity, idx = s_norm.max(0)
    s = immediate_sensitivity.detach().numpy().item()
    
    # (5) gradient of (4) - this is "beta" for linear-like models
    beta_vec = torch.autograd.grad(immediate_sensitivity, inp, retain_graph=True)[0]
    #print(beta_vec)
    beta_max = beta_vec[idx].norm(p=2) # CHANGE
    #print(beta_max)
    final_sens = immediate_sensitivity + beta_max
    
    '''
    if epoch > 5:
        print(f"inputs: ",inp)
        print(f"outputs: ", outputs)
        print(f"loss: ", loss)
        print(f"first_order_grads: ", first_order_grads)
        print(f"grad_l2_norm:: ", grad_l2_norm)
        print(f"sensitivity_vec: ", sensitivity_vec)
        print(f"sensitivies: ", s)
    '''

    loss.backward()
    return loss, final_sens.detach().numpy().item(), beta_max.numpy().item()

In [189]:
def accuracy(model, X, y):
    Xt = torch.from_numpy(X).float()
    yt = torch.from_numpy(y).long()
    outputs = model(Xt)
    values, indices = outputs.max(dim=1)
    y_hat = indices.detach().numpy()
    accuracy = np.sum(y_hat == y) / len(y)
    return accuracy

In [190]:
def get_eps(epsilon, alpha, delta):
    ed_eps = epsilon + np.log(1/delta)/(alpha - 1)
    print(f'Total epsilon = {ed_eps}, delta = {delta}')
    return ed_eps

In [191]:
def run_experiment(epsilon, epochs, add_noise=False):
    # reset the model
    model = Classifier(n_features=n_features)
    model_criterion = nn.NLLLoss() 
    model_optimizer = optim.Adam(model.parameters(),lr=0.001)

    # parameters for Renyi differential privacy
#     omega = 10
#     rho_iter = rho / epochs
#     delta = 1e-5
#     eps = rho + 2*np.sqrt(rho * np.log(1/delta))
#     print(f'Total epsilon = {eps}, delta = {delta}')

    # parameters for tCDP differential privacy
    alpha = 200
    epsilon_iter = epsilon / epochs

    for epoch in range(epochs):
        for x_batch_train, y_batch_train in train_loader:
            model_optimizer.zero_grad()
            loss, batch_sensitivity, beta = grad_immediate_sensitivity(model, model_criterion, x_batch_train, y_batch_train,epoch)

            imm_sens = batch_sensitivity# / BATCH_SIZE
            # this is the scale of the Gaussian noise to be added to the batch gradient
            
            # For tCDP and smooth sensitivity
            # big_x = beta**2 / (4 * (1-(omega * (1 - np.exp(-beta))))**2)
            # sigma_sq = 2*(1-(omega * (1-np.exp(-beta)))) / (rho_iter - big_x)

        
            # For RDP and smooth sensitivity
            # calculating renyi divergence directly
            t = beta
            gamma = alpha * (np.exp(t) - 1) + 1
            sigma_sq = 2*gamma**2 * imm_sens**2 / (4* epsilon_iter * gamma**2 - alpha * t**2)
            sigma = np.sqrt(sigma_sq)

            #print('Smooth:', sigma)

            # Assuming IS is a bound on GS (no smooth sensitivity)
            sigma = np.sqrt(((batch_sensitivity/BATCH_SIZE)**2 * alpha) / (2 * epsilon_iter))

            #print('Global:', sigma)

            if add_noise:
                with torch.no_grad():
                    for p in model.parameters():
                        p.grad += (sigma * torch.randn(1).float())

            model_optimizer.step()

    return model

In [192]:
model = run_experiment(.001, 10, False)
accuracy(model, X_test, y_test)

/home/jnear/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in sqrt


0.95

In [193]:
model = run_experiment(1, 10, True)
accuracy(model, X_test, y_test)

0.79

In [194]:
def one_experiment(epsilon):
    model = run_experiment(epsilon, 10, True)
    return accuracy(model, X_test, y_test)

In [195]:
def run_experiments():
    epsilons = [0.01, 0.1, 1.0, 10.0, 100.0]
    runs = 10
    alpha = 200
    results = {}
    
    for eps in epsilons:
        ed_eps = get_eps(eps, 200, 1e-5)
        results[ed_eps] = [one_experiment(eps) for _ in range(runs)]
    
    return results

In [196]:
all_results = run_experiments()

Total epsilon = 0.06785389680889561, delta = 1e-05
Total epsilon = 0.1578538968088956, delta = 1e-05
Total epsilon = 1.0578538968088955, delta = 1e-05
Total epsilon = 10.057853896808895, delta = 1e-05
Total epsilon = 100.0578538968089, delta = 1e-05


In [197]:
setting = 'ours'

In [199]:
print(f'{setting}_epsilons = {list(all_results.keys())}')
print(f'{setting}_means = {[np.mean(vs) for vs in all_results.values()]}')
print(f'{setting}_stds = {[np.std(vs) for vs in all_results.values()]}')

ours_epsilons = [0.06785389680889561, 0.1578538968088956, 1.0578538968088955, 10.057853896808895, 100.0578538968089]
ours_means = [0.665, 0.8099999999999999, 0.8720000000000001, 0.9195, 0.9295]
ours_stds = [0.1267675037223657, 0.12599603168354154, 0.055551777649324625, 0.013499999999999995, 0.005220153254455257]
